<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/CMD_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
samples_iter = 200
dropout = 0.1
image_size = 256

In [3]:
!pip install astro-datasets --upgrade
!pip install tensorflow_datasets --upgrade
import astro_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
ds_train, info_train = tfds.load(name='cmd', split='train[:90%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot', 'parameters': ['omegam']})
ds_valid, info_valid = tfds.load(name='cmd', split='train[90%:95%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})
ds_test, info_test = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})

In [5]:
minimum = np.array([0.1, 0.6])
maximum = np.array([0.5, 1.0])  

def resize(image, label, size):
  image = tf.image.resize(image, size=[size, size])
  return image, label

#ds_train = ds_train.map(lambda image, label: resize(image, label, image_size))
#ds_valid = ds_valid.map(lambda image, label: resize(image, label, image_size))
#ds_test = ds_test.map(lambda image, label: resize(image, label, image_size))

def normalize(image, label):
  image = tf.math.log(image)
  label = (label - minimum[0])/(maximum[0] - minimum[0])
  return image, label

ds_train = ds_train.map(normalize)
ds_valid = ds_valid.map(normalize)
ds_test = ds_test.map(normalize)

def rotate_map(image, label):
  return tf.image.rot90(image, np.random.choice([-1,0,1,2])), label

#ds_train = ds_train.map(rotate_map)

In [6]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(image_size, image_size, 1)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(120, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(dropout),
    tf.keras.layers.Dense(1, activation='linear'),
    ])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 6)       156       
                                                                 
 re_lu (ReLU)                (None, 256, 256, 6)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 6)      0         
 )                                                               
                                                                 
 monte_carlo_dropout (MonteC  (None, 127, 127, 6)      0         
 arloDropout)                                                    
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 16)      2416      
                                                                 
 re_lu_1 (ReLU)              (None, 127, 127, 16)      0

In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

In [10]:
model.compile(optimizer=opt,
              loss='MeanSquaredError',
              metrics='mae')

In [11]:
checkpoint_path = "cmd/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [12]:
model.fit(ds_train, epochs=200, callbacks=[reduce_lr, cp_callback, es_callback], validation_data=ds_valid)

Epoch 1/200
180/180 [==============================] - ETA: 0s - loss: 98.1577 - mae: 1.9178
Epoch 1: val_loss improved from inf to 0.13300, saving model to cmd/cp.ckpt
180/180 [==============================] - 20s 91ms/step - loss: 98.1577 - mae: 1.9178 - val_loss: 0.1330 - val_mae: 0.2976 - lr: 1.0000e-04
Epoch 2/200
179/180 [============================>.] - ETA: 0s - loss: 0.1351 - mae: 0.3014
Epoch 2: val_loss did not improve from 0.13300
180/180 [==============================] - 13s 69ms/step - loss: 0.1352 - mae: 0.3015 - val_loss: 0.1499 - val_mae: 0.3193 - lr: 1.0000e-04
Epoch 3/200
180/180 [==============================] - ETA: 0s - loss: 0.1362 - mae: 0.3004
Epoch 3: val_loss did not improve from 0.13300
180/180 [==============================] - 13s 73ms/step - loss: 0.1362 - mae: 0.3004 - val_loss: 0.1368 - val_mae: 0.3023 - lr: 1.0000e-04
Epoch 4/200
180/180 [==============================] - ETA: 0s - loss: 0.1380 - mae: 0.3039
Epoch 4: val_loss did not improve from 0

In [13]:
new_model = tf.keras.models.load_model('cmd/cp.ckpt')

In [14]:
def unnormalize(image, label):  
  image = tf.math.exp(image)
  label = (label*(maximum[0] - minimum[0])) + minimum[0]
  return image, label

In [15]:
mapes = []
for _ in range(samples_iter):
  y = []
  yhat = []
  for (x_batch, y_batch) in ds_test:
      yhat.append(new_model.predict(x_batch))
      y.append(y_batch)
  y = np.concatenate(y)
  yhat = np.concatenate(yhat)
  # Unnormalise 
  y = (y*(maximum[0] - minimum[0])) + minimum[0]
  yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
  print('MAPE', 100*np.mean(np.abs((yhat-y)/y)))
  mapes.append(100*np.mean(np.abs((yhat-y)/y)))

MAPE 25.322015197491876
MAPE 24.872940307109857
MAPE 24.88861657790554
MAPE 24.373591602241625
MAPE 24.700392500567038
MAPE 24.249254433880324
MAPE 24.77414095856304
MAPE 24.594725216789918
MAPE 24.897451547116518
MAPE 24.199000184931645
MAPE 24.303375989064648
MAPE 24.39399674664598
MAPE 24.330250280713308
MAPE 24.962656942243864
MAPE 24.833808537969052
MAPE 25.244222109460846
MAPE 24.95193670759485
MAPE 25.115589278081334
MAPE 24.17963529709954
MAPE 24.19210366050158
MAPE 24.529434939284034
MAPE 24.132813561435686
MAPE 24.259496438703096
MAPE 25.026438533324953
MAPE 25.10202540323024
MAPE 24.587470616310476
MAPE 24.458055473897076
MAPE 24.17625968470022
MAPE 24.723946767422703
MAPE 24.930905396193833
MAPE 24.90879004433152
MAPE 24.69081038919058
MAPE 24.399267063212037
MAPE 23.587141796854706
MAPE 25.195110127141874
MAPE 24.219200222390615
MAPE 24.674955034405745
MAPE 24.672443187178043
MAPE 24.444903282131627
MAPE 25.012386572991417
MAPE 24.75695017748446
MAPE 24.088379382773688
MAP

In [16]:
print('MAPE', np.mean(mapes), np.std(mapes))

MAPE 24.646423900368582 0.37393828216280334


In [17]:
ds_test_S, info_test_S = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'SIMBA', 'field': 'Mtot','parameters': ['omegam']})
ds_test_S = ds_test_S.map(normalize)

In [18]:
mapes = []
for _ in range(samples_iter):
  y = []
  yhat = []
  for (x_batch, y_batch) in ds_test_S:
      yhat.append(new_model.predict(x_batch))
      y.append(y_batch)
  y = np.concatenate(y)
  yhat = np.concatenate(yhat)
  # Unnormalise 
  y = (y*(maximum[0] - minimum[0])) + minimum[0]
  yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
  print('MAPE', 100*np.mean(np.abs((yhat-y)/y)))
  mapes.append(100*np.mean(np.abs((yhat-y)/y)))

MAPE 26.296298853744787
MAPE 26.060249519139262
MAPE 25.960245144512147
MAPE 25.952162098733755
MAPE 26.269946216543083
MAPE 26.770404066017473
MAPE 26.14374904162559
MAPE 26.460231512330928
MAPE 26.5346341992521
MAPE 25.929587170541645
MAPE 26.21358877773205
MAPE 26.7743915210752
MAPE 26.630107090598003
MAPE 26.591740098998244
MAPE 26.29011733742328
MAPE 26.280443605206372
MAPE 26.558476735140008
MAPE 26.80036998000891
MAPE 26.15332996072223
MAPE 26.917651210212075
MAPE 26.16368225954305
MAPE 26.575307015273626
MAPE 26.371386218758243
MAPE 26.49088723017802
MAPE 26.416718085445268
MAPE 25.780894526715812
MAPE 26.208645022907568
MAPE 26.152012046265018
MAPE 26.682450770160198
MAPE 26.259403404809632
MAPE 25.779977233147815
MAPE 26.04188638086345
MAPE 26.140704419492483
MAPE 26.03640844079336
MAPE 26.213304834464406
MAPE 25.811844528766166
MAPE 25.91524758517844
MAPE 26.650465536906125
MAPE 26.336304528718944
MAPE 25.77018416102416
MAPE 26.397643980206535
MAPE 25.665110430563733
MAPE 26

In [19]:
print('MAPE', np.mean(mapes), np.std(mapes))

MAPE 26.377535113262844 0.3411876758311678
